In [2]:
import pandas as pd

In [3]:
import itertools
def crossProduct(files,df1,df2):
    CPdf = pd.DataFrame(columns= list(files[0][:-3]+df1.columns)+list(files[1][:-3]+df2.columns),index=range(df1.shape[0]*df2.shape[0]))
    i,j,k=0,0,0
    for i, j in itertools.product(range(df1.shape[0]), range(df2.shape[0])):
        CPdf.iloc[k] = list(df1.iloc[i])+list(df2.iloc[j])
        k+=1
    
    return CPdf

In [4]:
def NaturalJoin(files,df1,df2):
    # Check common attributes
    comm_attr=[]
    for attributes in list(df1.columns):
        if attributes in list(df2.columns):
            comm_attr.append(attributes)

    if len(comm_attr) == 0:
        return crossProduct(files,df1,df2)
    else:
        df2.drop(comm_attr,axis=1).to_csv('after_drop.csv',index=False)
        temp = pd.read_csv('after_drop.csv')
        result_attri = (list(files[0][:-3]+df1.columns)) + (list(files[1][:-3]+temp.columns))
        nj_df = pd.DataFrame(columns = result_attri,index = range(df1.shape[0]*temp.shape[0]))

        k=0
        for i in range(df1.shape[0]):
            for j in range(df2.shape[0]):
                if set(df1.loc[i,comm_attr]) == set(df2.loc[j,comm_attr]):    
                    nj_df.loc[k] = (list(df1.loc[i]) + list(temp.loc[j]))
                    # print(nj_df.loc[k])
                    k+=1

        # Extra rows drop it!!
        nj_df.drop(range(k,df1.shape[0]*temp.shape[0]),axis=0,inplace=True)

        return nj_df
        
    

In [6]:
# query = input('Give sql query')
query = """SELECT Name.Age FROM Name.csv CROSS PRODUCT own.csv WHERE Name.Age == 24"""
query2 = """SELECT * FROM bank_1.csv Natural Join bank_2.csv WHERE bank_2.age > 30 ORDER BY bank_1.job ASC"""

In [7]:
def sqlParser(q):
    keyWords = q.split(' ')
    """
    Selecting the Attributes 
    """
    star = '*'
    starValue = False
    attributes=[]
    if star in keyWords:
        starValue = True
    else:
        attributes = keyWords[1:keyWords.index('FROM')]

    """
    Selecting the .csv files from the query
    """
    i=0
    dotCSV = '.csv'
    filename = [keyWords[i] for i in range(len(keyWords)) if dotCSV in keyWords[i]]
    """
    Selecting the WHERE clause
    """
    where = "WHERE"
    whereClause= []
    orderby = []
    if where in q:
        indexWhere = keyWords.index("WHERE")
        whereClause = keyWords[1+indexWhere:]

        ordby = 'BY'
        if ordby in whereClause:
            indexorderby = whereClause.index("BY")
            orderclause = whereClause[indexorderby+1:]
    return keyWords,starValue, attributes, filename, whereClause,orderclause

In [9]:
def order_by(orderby,df):
    df = df.astype(str)
    if ( orderby[1] == 'ASC'):
        df.sort_values(by=orderby[0], inplace = True, ascending=True)
    elif (orderby[1] == 'DESC'): 
        df.sort_values(by=orderby[0], inplace = True, ascending=False)
        df.to_csv('./order.csv',index=False)
    return df

In [10]:
def wherefunc(where,df):

    op = where[1]
    attri = where[0]
    where_df = pd.DataFrame(columns=list(df.columns),index=range(df.shape[0]))
    df = df.astype(str)
    value = where[2]
    
    k=0
    for i in range(df.shape[0]):
        if op == '<=':
            if df.loc[i,attri] <= value:
                where_df.loc[k] = list(df.loc[i])
                k+=1
        elif op == '=':
            if df.loc[i,attri] == value:
                where_df.loc[k] = list(df.loc[i])
                k+=1
        elif op == '>':
            if df.loc[i,attri] > value:
                where_df.loc[k] = list(df.loc[i])
                k+=1
        elif op == '>=':
            if df.loc[i,attri] >= value:
                where_df.loc[k] = list(df.loc[i])
                k+=1
        elif df.loc[i,attri] < value:
            where_df.loc[k] = list(df.loc[i])
            k+=1        
    where_df.drop(range(k,df.shape[0]),inplace=True)
    return where_df

In [11]:
key,s,attr,files,where,order = sqlParser(query2)

df1 = pd.read_csv(files[0])
df2 = pd.read_csv(files[1])
# From part
if 'CROSS' in key:
    result = crossProduct(files,df1,df2)
else:
    result = NaturalJoin(files,df1,df2)

if len(where)!=0:
    out = wherefunc(where,result)

if len(order)!=0:
    out = order_by(order,out)
# if all attribute
if s :
     out.to_csv('output.csv',index=False)
# only selected attributes
else:
    out.to_csv('output.csv',columns=attr,index=False)
# out.head()